## Collecting song information

In [1]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 kB 3.5 MB/s eta 0:00:00a 0:00:01


In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
cid = "3519692942004325a2c7160c90717ca5"
secret = "9243be1df96e48bb829c4b07254bd82c"

In [4]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [10]:
playlist_link = "https://open.spotify.com/playlist/3ZcAJLSa7T9QpqCgIG2MAV?si=sLtQlReoRViuVhcpkhOqtQ&dd=1&nd=1"
uri = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(uri)["items"]]

In [12]:
print(sp.audio_features(track_uris)[0])

{'danceability': 0.295, 'energy': 0.176, 'key': 2, 'loudness': -13.949, 'mode': 0, 'speechiness': 0.0397, 'acousticness': 0.849, 'instrumentalness': 0.659, 'liveness': 0.105, 'valence': 0.147, 'tempo': 120.47, 'type': 'audio_features', 'id': '3OCxOUUH3FUf8xk0RuWADJ', 'uri': 'spotify:track:3OCxOUUH3FUf8xk0RuWADJ', 'track_href': 'https://api.spotify.com/v1/tracks/3OCxOUUH3FUf8xk0RuWADJ', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/3OCxOUUH3FUf8xk0RuWADJ', 'duration_ms': 183570, 'time_signature': 4}


In [15]:
import pandas as pd

# Initialize empty lists for columns
song_names = []
danceability = []
energy = []
key = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []

# Loop through the tracks and append the data to the respective lists
for track in sp.playlist_tracks(playlist_link)["items"]:
    track_name = track["track"]["name"]
    song_names.append(track_name)
    
    audio_features = sp.audio_features(track["track"]["uri"])[0]
    danceability.append(audio_features["danceability"])
    energy.append(audio_features["energy"])
    key.append(audio_features["key"])
    loudness.append(audio_features["loudness"])
    mode.append(audio_features["mode"])
    speechiness.append(audio_features["speechiness"])
    acousticness.append(audio_features["acousticness"])
    instrumentalness.append(audio_features["instrumentalness"])
    liveness.append(audio_features["liveness"])
    valence.append(audio_features["valence"])
    tempo.append(audio_features["tempo"])

# Create a DataFrame
data = {
    "Song Name": song_names,
    "Danceability": danceability,
    "Energy": energy,
    "Key": key,
    "Loudness": loudness,
    "Mode": mode,
    "Speechiness": speechiness,
    "Acousticness": acousticness,
    "Instrumentalness": instrumentalness,
    "Liveness": liveness,
    "Valence": valence,
    "Tempo": tempo
}

df = pd.DataFrame(data)

In [16]:
df

,Song Name,Danceability,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,ザ・ワード II,0.295,0.176,2,-13.949,0,0.0397,0.849000,0.659000,0.1050,0.147,120.470
1,Who Dares To Believe In Me?,0.531,0.875,2,-3.725,1,0.0915,0.001460,0.512000,0.2400,0.500,120.940
2,Sweet Love,0.504,0.728,10,-5.699,0,0.0234,0.360000,0.000000,0.1590,0.683,89.255
3,Ultimate High,0.836,0.939,9,-5.274,0,0.1500,0.026000,0.880000,0.1770,0.586,127.992
4,In Love With You,0.445,0.903,1,-5.824,1,0.0632,0.000134,0.025700,0.6990,0.126,121.016
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Poison Arrow,0.736,0.876,11,-5.894,0,0.0839,0.182000,0.007760,0.1190,0.802,125.843
96,Drum Workout!,0.787,0.835,9,-9.461,1,0.0675,0.118000,0.870000,0.3660,0.228,125.998
97,ADD SUV (feat. Pharrell Williams) - Armand Van...,0.843,0.972,6,-6.580,0,0.1440,0.003260,0.301000,0.0324,0.496,124.989
98,Sternenhimmel,0.577,0.911,6,-8.914,0,0.1190,0.047100,0.000000,0.0862,0.195,133.979


In [ ]:
from sklearn.preprocessing import MinMaxScaler

# we don't need to normalize mode because it is always 0 or 1

# List of features to normalize
normalize_metrics = [
    "Danceability",
    "Energy",
    "Key",
    "Loudness",
    "Speechiness",
    "Acousticness",
    "Instrumentalness",
    "Liveness",
    "Valence",
    "Tempo",
]

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Normalize the features
df[normalize_metrics] = scaler.fit_transform(df[features_to_normalize])

TypeError: Passing a dict as an indexer is not supported. Use a list instead.

In [ ]:
df

# Score

We need a weighting system that identifies the importance of each feature. It should add up to 1.

In [24]:
def get_relative_key(key, mode):
    if mode == 1:  # Major key
        return (key + 9) % 12  # Relative minor key
    else:  # Minor key
        return (key + 3) % 12  # Relative major key

def key_compatibility(key1, mode1, key2, mode2):
    if key1 == key2:
        return True
    if mode1 != mode2 and (key1 == get_relative_key(key2, mode2) or key2 == get_relative_key(key1, mode1)):
        return True
    return False

def evaluate_transition(song1, song2):
    # Weights for different attributes
    weights = {
        'danceability': 1,
        'energy': 1,
        'loudness': 0.5,
        'tempo': 10,  # Increase the weight of tempo
        'valence': 1
    }

    # Check key compatibility
    if not key_compatibility(song1['key'], song1['mode'], song2['key'], song2['mode']):
        return float('inf')  # Return a large score if keys are not compatible

    # Calculate the differences in attributes
    diff = {}
    for attribute in weights:
        diff[attribute] = abs(song1[attribute] - song2[attribute])

    # Normalize loudness differences
    diff['loudness'] /= 60  # Assume max difference is 60 dB

    # Calculate tempo difference considering double/half time mixing
    tempo_diff = min(
        abs(song1['tempo'] - song2['tempo']),
        abs(song1['tempo'] * 2 - song2['tempo']),
        abs(song1['tempo'] / 2 - song2['tempo'])
    )
    diff['tempo'] = tempo_diff / 200  # Normalize tempo difference

    # Calculate the transition score
    score = 0
    for attribute in weights:
        score += weights[attribute] * diff[attribute]

    return score